# Reaction Engineering Lesson: PFR vs. CSTR Performance

**Objective:** This lesson will compare the performance of the two most common ideal reactor types: the Continuous Stirred-Tank Reactor (CSTR) and the Plug Flow Reactor (PFR). We will simulate both for a simple reaction to determine which is more efficient.

**Learning Goals:**
1.  Review the design equations for both CSTR and PFR models.
2.  Understand the fundamental difference in their mixing models.
3.  Implement both models in Python to calculate reactant conversion as a function of residence time.
4.  Visually and quantitatively compare the performance of the two reactor types.
5.  Explain *why* one reactor type typically yields higher conversion for the same volume.

## The Models: Reactor Design Equations

We will consider a simple, irreversible, nth-order liquid-phase reaction: $A \rightarrow B$ with a rate law of $r = k C_A^n$.

### 1. The CSTR (Continuous Stirred-Tank Reactor)
A CSTR assumes perfect mixing, meaning the concentration inside the reactor is uniform and equal to the outlet concentration. The design equation is an **algebraic equation** derived from a mass balance:
$$ \tau_{CSTR} = \frac{V}{F_0} = \frac{C_{A0} - C_A}{k C_A^n} $$
where $\tau$ is the residence time.

### 2. The PFR (Plug Flow Reactor)
A PFR assumes no axial mixing, but perfect radial mixing. Fluid flows like a "plug". The model is a **differential equation** derived from a balance over a differential slice of the reactor:
$$ \frac{dC_A}{d\tau} = -k C_A^n $$

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# --- Reaction and Feed Parameters ---
k = 0.5           # Reaction rate constant (L/mol.min for n=2)
n = 2.0           # Reaction order
C_A0 = 1.0        # Initial concentration of A (mol/L)

# Define a range of residence times to simulate
tau_max = 10.0      # minutes
tau_range = np.linspace(0.01, tau_max, 100)

# --- PFR Model Function ---
def pfr_model(tau, C_A):
    """Defines the PFR ODE."""
    return -k * (C_A**n)

print("Parameters and PFR model are defined.")

## Simulation and Comparison

We will now solve the PFR model using an ODE solver and calculate the CSTR performance using its algebraic equation. We will then plot conversion for both.

In [ ]:
# --- PFR Simulation ---
tau_span = (0, tau_max)
C_A_initial = [C_A0]
pfr_solution = solve_ivp(pfr_model, tau_span, C_A_initial, dense_output=True, t_eval=tau_range)
C_A_pfr = pfr_solution.y[0]
X_pfr = (C_A0 - C_A_pfr) / C_A0 # PFR Conversion

# --- CSTR Calculation ---
# We need to solve the algebraic CSTR equation for C_A at each tau.
# This can be complex for n!=1. For n=2: k*tau*C_A^2 + C_A - C_A0 = 0
C_A_cstr = []
for tau in tau_range:
    # Using the quadratic formula to solve for C_A
    # a*x^2 + b*x + c = 0 -> x = (-b + sqrt(b^2 - 4ac)) / 2a
    a = k * tau
    b = 1
    c = -C_A0
    C_A = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
    C_A_cstr.append(C_A)

C_A_cstr = np.array(C_A_cstr)
X_cstr = (C_A0 - C_A_cstr) / C_A0 # CSTR Conversion

print("Simulations for both reactors are complete.")

In [ ]:
# --- Plotting the Comparison ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 7))

plt.plot(tau_range, X_pfr * 100, 'b-', linewidth=3, label='PFR')
plt.plot(tau_range, X_cstr * 100, 'r--', linewidth=3, label='CSTR')

# Formatting
plt.title('PFR vs. CSTR Performance Comparison (n=2)', fontsize=16, weight='bold')
plt.xlabel('Residence Time, $\tau$ (minutes)', fontsize=14)
plt.ylabel('Conversion, X (%)', fontsize=14)
plt.legend(fontsize=12)
plt.xlim(0, tau_max)
plt.ylim(0, 100)
plt.grid(True)
plt.show()

### Interpreting the Plot

The conclusion is clear and immediate: **For any given residence time (reactor volume), the PFR achieves a significantly higher conversion than the CSTR.**

**Why?**
The reaction rate depends on concentration ($r = k C_A^n$). 
*   In a **PFR**, the concentration of reactant A is high at the inlet and gradually decreases. This means the average reaction rate along the reactor is high.
*   In a **CSTR**, the moment the feed enters, it is instantly mixed and diluted down to the low outlet concentration. The entire reaction takes place at this low concentration, resulting in a much lower reaction rate.

This simulation visually proves that preserving high reactant concentration, as a PFR does, is more efficient than the complete back-mixing of a CSTR.

## Student Challenges

1.  **First-Order Reaction:** Change the reaction order `n` to `1`. Is the PFR still more efficient? (Yes, it will be). You will also need to change the CSTR calculation to the simpler form: $C_A = C_{A0} / (1 + k\tau)$.
2.  **Reactors in Series:** A classic concept is that multiple CSTRs in series can begin to approximate a PFR. Can you simulate two identical CSTRs in series and plot their final conversion on the same graph? The outlet of the first CSTR becomes the inlet of the second. You should find the result lies between the single CSTR and the PFR curves!